In [2]:
import numpy as np
import pandas as pd

## Problem 1. 

Define a CorruptQueue class that contains attributes RegularQueue and VIPStack, which is a 
representation of a queue and a stack, respectively. The constructor reads its data from a text file called 
officeinput.txt. This text file contains information on what CorruptQueue should do a line at a time. 
As CorruptQueue reads a line, it should process the information and printout a corresponding output on the 
command prompt. 

## Problem 2. 

Create a simulation that asks for a text file input from a user (which will be the file used in the 
constructor in the CorruptQueue class. There will just be one action per line, and you can assume that each 
line is always correct.  The simulation will then output all the output lines in a separate text file (also to be 
inputted by the user). Sample input and output files will be provided for reference. 

In [189]:
# CorruptQueue class is based on the requirements for Problem 2.
class CorruptQueue:
    
    def __init__(self, inpfile, outfile):
        
        # Initialize Queuing process attributes
        self.SupervisorPresent = False
        self.RegularQueue = []
        self.VIPStack = []
        
        # Read input file
        self.inpfile = open(inpfile, 'r')
        self.inpactions = [s.split(",") for s in [s.replace("\n", "") for s in self.inpfile.readlines()]]
        self.outfile = open(outfile, 'w')
        # Interpret actions in input file and output to file
        for s in self.inpactions:
            if s[0] == 'lineup':
                self.lineup(s)
            elif s[0] == 'serve':
                self.serve(s)
            elif s[0] == 'arrive':
                self.arrive(s)
            elif s[0] == 'leave':
                self.leave(s)
        
        # Close file after generating result
        self.inpfile.close()
        self.outfile.close()
        
    def lineup(self, s):
        if self.SupervisorPresent == True:
            # if supervised, all clients line up in RegularQueue
            print('{1} client {0} lines up at RegularQueue'.format(s[1],s[2]), file=self.outfile)
            self.RegularQueue.append(s[1])
        else:
            if s[2] == 'VIP':
                # if unsupervised, clients line up in separate VIP and regular lines
                print('{1} client {0} lines up at VIPStack'.format(s[1],s[2]), file=self.outfile)
                self.VIPStack.append(s[1])
            else:
                print('{1} client {0} lines up at RegularQueue'.format(s[1],s[2]), file=self.outfile)
                self.RegularQueue.append(s[1])
    def serve(self, s):
        if len(self.RegularQueue) == 0 and len(self.VIPStack) == 0:
            # edge case if 'serve' command is executed with none in queue
            print('No one in queue', file=self.outfile)
        else:
            if self.SupervisorPresent == True or len(self.VIPStack) == 0:
                # serve first in line at RegularQueue
                name = self.RegularQueue[0]
                print('Now serving {} from RegularQueue'.format(name), file=self.outfile)
                self.RegularQueue = self.RegularQueue[1:]
            else:
                # serve latest VIP to arrive if there are people in VIPStack
                name = self.VIPStack[-1]
                print('Now serving {} from VIPStack'.format(name), file=self.outfile)
                self.VIPStack = self.VIPStack[:-1]
    def arrive(self,s):
        # VIPStack appended to RegularQueue, then emptied
        print('Supervisor present', file=self.outfile)
        self.SupervisorPresent = True
        self.RegularQueue += self.VIPStack
        self.VIPStack = []

    def leave(self, s):
        print('Supervisor not here', file=self.outfile)
        self.SupervisorPresent = False

In [190]:
# For Problem 1, output file may be read in command prompt with the ff code
Sample = CorruptQueue('officeinput.txt', 'officeoutput.txt')
with open('officeoutput.txt','r') as Sampleout:
    print(Sampleout.read())

regular client John lines up at RegularQueue
regular client Bob lines up at RegularQueue
regular client Tom lines up at RegularQueue
VIP client Sarah lines up at VIPStack
VIP client Marie lines up at VIPStack
VIP client Joan lines up at VIPStack
Now serving Joan from VIPStack
Now serving Marie from VIPStack
Supervisor present
Now serving John from RegularQueue
Now serving Bob from RegularQueue
Now serving Tom from RegularQueue
VIP client Bea lines up at RegularQueue
regular client Hank lines up at RegularQueue
Now serving Sarah from RegularQueue
Now serving Bea from RegularQueue
Now serving Hank from RegularQueue
Supervisor not here
regular client Art lines up at RegularQueue
VIP client Daisy lines up at VIPStack
regular client Marius lines up at RegularQueue
VIP client Dane lines up at VIPStack
Now serving Dane from VIPStack
Supervisor present
Now serving Art from RegularQueue
Now serving Marius from RegularQueue
Now serving Daisy from RegularQueue
Supervisor not here



## Problem 3. 

Add the following attributes to the CorruptQueue: 
 
     - lambda – a numeric value that represents the distribution of people lining up the queue, or the average number of people lining up per minute 

     - mu – a numeric value that represents the mean or average service time 
     
     - sigma – a numeric value that represents the standard deviation of the service time 
 
Provide get() and set() methods as well for these new attributes. 
 
Instead of reading input from a text file in the previous problem, create a class called CQSimulation that 
first  asks  for  a  positive  integer  indicating  the  number  of  times  the  simulation  will  occur.  Then,  for  each 
iteration of the simulation, values for the three new attributes will be entered, and the results temporarily 
stored.  Using the results of the calculations of the simulation, display the following: 
 
     - average wait time 
     
     - average time a customer is in the system 
 
In this simulation, the assumption is that the supervisor is ALWAYS present, meaning all customers, VIP 
or regular, fall in line in a single queue. 

In [182]:
# Based on
# B. Jain, “Simulating a queuing system in Python,” Medium, 26-Apr-2021. [Online]. 
# Available: https://towardsdatascience.com/simulating-a-queuing-system-in-python-8a7d1151d485. [Accessed: 12-May-2022]. 
class CorruptQueue3:
    def __init__(self, p = {'lambda':1,'mu':1,'sigma':0.2}, outfile ='', seed = 1 ):
        
        # Initialize output file, random distribution variables and seed for random distribution
        self.p = p
        if outfile != '':
            self.outfile = open(outfile, 'w')
        else:
            self.outfile = ''
        self.seed = seed
        np.random.seed(self.seed)
        
        # Initialize clock and initial event conditions
        self.clock = 0.0 #simulation clock
        self.lineupTime = 0.0 # time next lineup occurs
        self.endserviceTime = float('inf') # time current service finishes, infinite if idle
        self.serverState = 'idle' # idle or busy
        self.queue = 0 #current queue
        
        # Initialize statistics
        self.lineups = [] # list of lineup timestamps
        self.services = [] # list of service timestamps
        self.avgWaitTime = 0 # average wait time
        self.avgTotalTime = 0 # average time a customer is in system
        
        # Other statistics implemented in the source material
        self.serveSum = 0 # sum of service time
        self.queueTotal = 0 # total no of customers who had to queue
        
        # Execute a single simulation consisting of 100 served customers
        while len(self.services) < 100:
            self.timeAdv()
            
        # Start of service is the end of service of the previous customer
        # Calculate wait time as 0 or difference of lineup time and previous serve time
        sumWaitTime = 0
        for i in range(1,100):
            sumWaitTime += max(0, self.services[i-1] - self.lineups[i])
        self.avgWaitTime = sumWaitTime / 100
        # Calculate total time as difference of lineup time and end of service time
        sumTotalTime = 0
        for i in range(1,100):
            sumTotalTime += max(0,self.services[i] - self.lineups[i])
        self.avgTotalTime = sumTotalTime / 100
        
    def timeAdv(self):
        # advance time to next event, whether time of next lineup or departure of current customer
        nextEvent = min(self.lineupTime, self.endserviceTime)
        # then, advance time
        self.clock = nextEvent
        
        # choose lineup or departure (end of service) event depending on which is sooner
        if self.lineupTime < self.endserviceTime:
            self.lineup()
        elif self.endserviceTime < self.lineupTime:
            self.endserve()
    
    def lineup(self):
        self.lineups.append(self.clock)
        if self.queue == 0:
            # if no one is in queue, but the server is busy, add 1 to queue
            if self.serverState == 'busy':
                if self.outfile != '':
                    print('New customer begins queue at {}'.format(np.round(self.clock,2)), file=self.outfile)
                self.queue += 1
                self.queueTotal += 1
                # generate next lineup time
                self.lineupTime = self.clock + self.gen_lineup()
            # else, customer is served: server switches from idle to busy
            else:
                if self.outfile != '':
                    print('New customer is served at {}'.format(np.round(self.clock,2)), file=self.outfile)
                self.serverState = 'busy'
                # generate service period, increment sum of service periods, generate next lineup and departure time
                self.serve = self.gen_serve()
                self.serveSum += self.serve
                self.endserviceTime = self.clock + self.serve
                self.lineupTime = self.clock + self.gen_lineup()
        else:
            # if there are people in queue, new customer lines up
            self.queue += 1
            self.queueTotal += 1
            if self.outfile != '':
                print('New customer lines up no. {1} at {0}'.format(np.round(self.clock,2), self.queue), file=self.outfile)
            # generate next lineup time
            self.lineupTime = self.clock + self.gen_lineup()
    
    def endserve(self):
        self.services.append(self.clock)
        # if people are in queue, next person in queue is served. generate service period and next departure time
        if self.queue > 0:
            if self.outfile != '':
                print('Customer departs, customer in queue no. {1} is served at {0}'.format(np.round(self.clock,2), self.queue), file=self.outfile)
            self.serve = self.gen_serve()
            self.serveSum += self.serve
            self.endserviceTime = self.clock + self.serve
            self.queue -= 1
        # else, server stays idle
        else:
            if self.outfile != '':
                print('Customer departs at {}'.format(np.round(self.clock,2)), file=self.outfile)
            self.endserviceTime = float('inf')
            self.serverState = 'idle'
    
    def gen_lineup(self):
        # from documentation: np.random.exponential input is 1/frequency
        return np.random.exponential(1/self.p['lambda'])
    
    def gen_serve(self):
        return np.random.normal(self.p['mu'], self.p['sigma'])
    
    def get(self, var):
        return self.p[var]

    def set(self, var, val):
        self.p[var] = val

In [183]:
# Show sample of a simulation
# Sample2 = CorruptQueue3({'lambda':1,'mu':0.9,'sigma':0.3}, 'simoutput.txt')
# with open('simoutput.txt','r') as Sample2out:
#    print(Sample2out.read())

In [184]:
# Show list of lineup, service times and calculation of statistics
#print(np.round(Sample2.lineups,2))
#print(np.round(Sample2.services,2))
#print(np.round(Sample2.avgWaitTime,2))
#print(np.round(Sample2.avgTotalTime,2))

In [188]:
# CQSimulation allows for running multiple CorruptQueue3 simulations
class CQSimulation:
    def __init__(self, iterations = 5, variables = None):
        # Initialize df to display information
        self.df = pd.DataFrame(columns=[
            'Average wait time',
            'Average total time in system'
        ])
        # Modify simulation variables per iteration if desired
        self.vars = variables
        
        for i in range(iterations):
            # Do not change default variables if none are specified
            if self.vars == None:
                s = CorruptQueue3(seed = i)
            # Else, take values as [lambda,mu,sigma]
            else:
                s = CorruptQueue3(p = {'lambda': self.vars[i][0],
                                       'mu': self.vars[i][1],
                                       'sigma': self.vars[i][2]}, seed = i)
            # Store calculated information to display and append to df
            a = pd.Series([
                s.avgWaitTime,
                s.avgTotalTime
            ], index=self.df.columns)
            self.df = self.df.append(a,ignore_index=True)

In [187]:
# Example without custom variables
Sample3 = CQSimulation()
Sample3.df

,Average wait time,Average total time in system
0,2.973303,3.978235
1,2.639776,3.655242
2,4.482408,5.458078
3,5.681828,6.664464
4,1.723792,2.717566


In [186]:
# Example with custom variables
Sample4 = CQSimulation(variables = [
    [1,1,0.2], # arrival and service equal
    [0.6,1,0.2], # slower arrival than service
    [1,1,0.5], # high standard deviation
    [5,0.1,0.05], # fast arrival and service
    [5,1,0.05], # fast arrival, slow service
])
Sample4.df

,Average wait time,Average total time in system
0,2.973303,3.978235
1,1.170332,2.174255
2,4.133359,5.071924
3,0.077673,0.179241
4,39.551153,40.546714
